# Project 1 Starter

**This is draft - version 0 - changes are possible and will be anounced.**

Project 1 is to allow students to practice Data Science concepts learned so far.

The project will include following tasks:
- Load dataset. Don't use "index" column for training.
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Validation/Test
- Add engineered features
- Train and tune ML model
- Provide final metrics using Test dataset

### Types of models to train

Your final submission should include single model. 
The model set you should try to come up with best model:
1. Sklearn Logistic Regression - try all combinations of regularization
2. H2O-3 GLM - try different combinations of regularization



### Feature engineering

You should train/fit categorical features scalers and encoders on Train only. Use `transform` or equivalent function on Validation/Test datasets.

It is important to understand all the steps before model training, so that you can reliably replicate and test them to produce scoring function.


You should generate various new features. Examples of such features can be seen in the Module-3 lecture on GLMs.
Your final model should have at least **10** new engineered features. On-hot-encoding, label encoding, and target encoding is not included in the **10** features.
You can try, but target encoding is not expected to produce improvement for Linear models.

Ideas for Feature engineering for various types of variables:
1. https://docs.h2o.ai/driverless-ai/1-10-lts/docs/userguide/transformations.html
2. GLM lecture and hands-on (Module-3)


**Note**: 
- You don't have to perform feature engineering using H2O-3 even if you decided to use H2O-3 GLM for model training.
- It is OK to perfor feature engineering using any technique, as long as you can replicate it correctly in the Scoring function.


### Threshold calculation

You will need to calculate optimal threshold for class assignment using F1 metric:
- If using sklearn, use F1 `macro`: `f1_score(y_true, y_pred, average='macro')` 
- If using H2O-3, use F1

You will need to find optimal probability threshold for class assignment, the threshold that maximizes above F1.



### Scoring function

The Project-1 will be graded based on the completeness and performance of your final model against the hold-out dataset.
The hold-out dataset will not be known to the students. As part of your deliverables, you will need to submit a scoring function. The scoring function will perform the following:
- Accept dataset in the same format as provided with the project, minus "MIS_Status" column
- Load trained model and any encoders/scalers that are needed to transform data
- Transform dataset into format that can be scored with the trained model
- Score the dataset and return the results, for each record
    - Record ID
    - Record label as determined by final model (0 or 1)
    - If your model returns probabilities, you need to assign the label based on maximum F1 threshold
    
Scoring function header:
```
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    l = data.shape[0]
    return l*[0]
```

Look for full example of scoring function at the bottom of the notebook. **Don't copy as is - this is just an example**



### Deliverables in a single zip file in the following structure:
- `notebook` (folder)
    - Jupyter notebook with complete code to manipulate data, train and tune final model. `ipynb` format
    - Jupyter notebook in `html` format
- `artifacts` (folder)
    - Model and any potential encoders in the "pkl" format or native H2O-3 format (for H2O-3 model)
    - Scoring function that will load the final model and encoders. Separate from above notebook or `.py` file



Your notebook should include explanations about your code and be designed to be easily followed and results replicated. Once you are done with the final version, you will need to test it by running all cells from top to bottom after restarting Kernel. It can be done by running `Kernel -> Restart & Run All`


**Important**: To speed up progress, first produce working code using a small subset of the dataset.

## Dataset description

The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

**Don't use original dataset, use only dataset provided with project requirements in eLearning**

## Preparation

Use dataset provided in the eLearning

In [205]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [206]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

### Load data

In [207]:
data = pd.read_csv('SBA_loans_project_1.zip')

In [208]:
print("Data shape:", data.shape)

Data shape: (809247, 21)


In [209]:
data

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,"$743,000.00",$0.00,"$743,000.00","$743,000.00",P I F
1,1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,"$137,000.00",$0.00,"$137,000.00","$109,737.00",P I F
2,2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,"$280,000.00",$0.00,"$280,000.00","$210,000.00",P I F
3,3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,"$144,500.00",$0.00,"$144,500.00","$122,825.00",P I F
4,4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,"$52,500.00",$0.00,"$52,500.00","$42,000.00",P I F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,"$749,000.00",$0.00,"$750,000.00","$750,000.00",P I F
809243,809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,"$75,000.00",$0.00,"$75,000.00","$67,500.00",P I F
809244,809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,"$216,900.00",$0.00,"$216,900.00","$162,675.00",P I F
809245,809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,"$75,000.00",$0.00,"$75,000.00","$60,000.00",P I F


In [210]:
# Removing the 'index' column from the data  
data = data.drop('index', axis=1)

In [211]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,"$743,000.00",$0.00,"$743,000.00","$743,000.00",P I F
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,"$137,000.00",$0.00,"$137,000.00","$109,737.00",P I F
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,"$280,000.00",$0.00,"$280,000.00","$210,000.00",P I F
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,"$144,500.00",$0.00,"$144,500.00","$122,825.00",P I F
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,"$52,500.00",$0.00,"$52,500.00","$42,000.00",P I F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,"$749,000.00",$0.00,"$750,000.00","$750,000.00",P I F
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,"$75,000.00",$0.00,"$75,000.00","$67,500.00",P I F
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,"$216,900.00",$0.00,"$216,900.00","$162,675.00",P I F
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,"$75,000.00",$0.00,"$75,000.00","$60,000.00",P I F


**Review dataset**

In [212]:
desc_df = describe_more(data)
desc_df

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,31320,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
1,State,51,object,2,3,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
2,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
3,Bank,5716,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
4,BankState,55,object,2,3,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
5,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
6,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
7,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
8,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 932}"
9,CreateJob,234,int64,1,4,"{0: 566148, 1: 56789, 2: 52162, 3: 25945, 4: 1..."


## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

Any additional clean-up as you find fit.

In [213]:
# Replace encode Na/Null values
data.fillna(0, inplace=True)
data.isnull().values.any()

False

In [214]:
# convert the strings styled as '$XXXX.XX' to float values
dollar_columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']

for col in dollar_columns:
  data[col] = [float(val[1:].replace(',', '')) for val in data[col].values]

In [215]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,743000.0,0.0,743000.0,743000.0,P I F
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000.0,0.0,137000.0,109737.0,P I F
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,280000.0,0.0,280000.0,210000.0,P I F
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,144500.0,0.0,144500.0,122825.0,P I F
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,52500.0,0.0,52500.0,42000.0,P I F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,749000.0,0.0,750000.0,750000.0,P I F
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,75000.0,0.0,75000.0,67500.0,P I F
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,216900.0,0.0,216900.0,162675.0,P I F
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,75000.0,0.0,75000.0,60000.0,P I F


In [216]:
data.dtypes

City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
DisbursementGross    float64
BalanceGross         float64
GrAppv               float64
SBA_Appv             float64
MIS_Status            object
dtype: object

In [217]:
# Convert MIS_Status to 0/1. Make value "CHGOFF" as 1
data['MIS_Status'] = (data['MIS_Status'] == 'CHGOFF').astype(int)

# Check the unique values in MIS_Status column to confirm
print(data['MIS_Status'].unique())

[0 1]


In [218]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,N,743000.0,0.0,743000.0,743000.0,0
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,N,137000.0,0.0,137000.0,109737.0,0
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,N,280000.0,0.0,280000.0,210000.0,0
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,Y,144500.0,0.0,144500.0,122825.0,0
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,Y,52500.0,0.0,52500.0,42000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,N,749000.0,0.0,750000.0,750000.0,0
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,Y,75000.0,0.0,75000.0,67500.0,0
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,N,216900.0,0.0,216900.0,162675.0,0
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,N,75000.0,0.0,75000.0,60000.0,0


In [219]:
data.dtypes

City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
DisbursementGross    float64
BalanceGross         float64
GrAppv               float64
SBA_Appv             float64
MIS_Status             int32
dtype: object

In [220]:
desc_df1 = describe_more(data)
desc_df1

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,31321,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
1,State,52,object,1,2,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
2,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
3,Bank,5717,object,1,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
4,BankState,56,object,1,2,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
5,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
6,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
7,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
8,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 1060}"
9,CreateJob,234,int64,1,4,"{0: 566148, 1: 56789, 2: 52162, 3: 25945, 4: 1..."


## Categorical and numerical variables encoding

Encode categorical variables using either one of the techniques below. Don't use LabelEncoder.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- WOE encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_woe


WOE encoder can be used with numerical variables too. 


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [221]:
import category_encoders as ce

# Identify categorical columns with less than 10 valid values and exclude 'MIS_Status'
categ_columns = [col for col in data.columns if data[col].dtype == 'object' and data[col].nunique() < 10 and col != 'MIS_Status']

# One-hot encoding for identified categorical columns
one_hot_encoder = ce.OneHotEncoder(cols=categ_columns, use_cat_names=True)
data = one_hot_encoder.fit_transform(data)

In [222]:
data

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc_N,LowDoc_Y,LowDoc_A,LowDoc_0,LowDoc_R,LowDoc_0#,LowDoc_S,LowDoc_C,LowDoc_1,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,GLEN BURNIE,MD,21060,"BUSINESS FINANCE GROUP, INC.",VA,811111,240,7,1.0,6,7,1,1,0,1,0,0,0,0,0,0,0,0,743000.0,0.0,743000.0,743000.0,0
1,WEST BEND,WI,53095,JPMORGAN CHASE BANK NATL ASSOC,IL,722410,240,20,1.0,0,0,1,0,N,1,0,0,0,0,0,0,0,0,137000.0,0.0,137000.0,109737.0,0
2,SAN DIEGO,CA,92128,UMPQUA BANK,OR,0,120,2,1.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,280000.0,0.0,280000.0,210000.0,0
3,WEBSTER,MA,1570,HOMETOWN BANK A CO-OPERATIVE B,MA,621310,84,7,1.0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,144500.0,0.0,144500.0,122825.0,0
4,JOPLIN,MO,64804,U.S. BANK NATIONAL ASSOCIATION,OH,0,60,2,2.0,0,0,1,0,N,0,1,0,0,0,0,0,0,0,52500.0,0.0,52500.0,42000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,BATON ROUGE,LA,70808,LOUISIANA CAP. CERT. DEVEL COM,LA,0,240,24,2.0,24,0,38605,0,N,1,0,0,0,0,0,0,0,0,749000.0,0.0,750000.0,750000.0,0
809243,GREENVILLE,SC,29611,SUNTRUST BANK,GA,0,72,5,1.0,0,0,1,0,N,0,1,0,0,0,0,0,0,0,75000.0,0.0,75000.0,67500.0,0
809244,ST. PAUL,MN,55114,SUNRISE BANKS NATL ASSOC,MN,424990,120,31,1.0,0,31,1,1,0,1,0,0,0,0,0,0,0,0,216900.0,0.0,216900.0,162675.0,0
809245,TUSCALOOSA,AL,35401,BK OF TUSCALOOSA A DIVISION O,AL,0,120,4,2.0,0,0,1,1,N,1,0,0,0,0,0,0,0,0,75000.0,0.0,75000.0,60000.0,0


In [202]:
enc_cols=['LowDoc_N','LowDoc_Y','LowDoc_A','LowDoc_0','LowDoc_R','LowDoc_0#','LowDoc_S','LowDoc_C','LowDoc_1']

In [223]:
import category_encoders as ce

# Identify columns for WOE encoding
woe_columns = [col for col in data.columns 
               if (data[col].dtype=='object' or data[col].dtype=='float64' or data[col].dtype=='int64') 
               and col not in categ_columns and col not in enc_cols and col != 'MIS_Status']

# Create WOE encoder object
woe_encoder = ce.WOEEncoder(cols=woe_columns)

# Fit and transform the data using the WOE encoder
data = woe_encoder.fit_transform(data, data['MIS_Status'])

In [224]:
woe_encoder

WOEEncoder(cols=['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'Term',
                 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
                 'FranchiseCode', 'UrbanRural', 'RevLineCr',
                 'DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv'])

In [225]:
# Append '_woe' to the column names of the WOE-encoded columns
data = data.add_suffix('_woe')

In [226]:
data

,City_woe,State_woe,Zip_woe,Bank_woe,BankState_woe,NAICS_woe,Term_woe,NoEmp_woe,NewExist_woe,CreateJob_woe,RetainedJob_woe,FranchiseCode_woe,UrbanRural_woe,RevLineCr_woe,LowDoc_N_woe,LowDoc_Y_woe,LowDoc_A_woe,LowDoc_0_woe,LowDoc_R_woe,LowDoc_0#_woe,LowDoc_S_woe,LowDoc_C_woe,LowDoc_1_woe,DisbursementGross_woe,BalanceGross_woe,GrAppv_woe,SBA_Appv_woe,MIS_Status_woe
0,0.003713,0.147356,0.721934,-5.225612,1.069163,-0.154111,-4.587808,-0.069010,-0.032451,-0.126084,0.440338,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-1.054077,0.000012,-0.936294,-2.140267,0
1,-0.417501,-0.413527,-0.139469,0.573283,0.295860,-0.239184,-4.587808,-0.423380,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.533543,0.000012,-0.616351,0.000000,0
2,-0.185339,0.063209,0.052503,0.545063,0.036336,-0.848126,-2.776769,0.197624,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.189182,1,0,0,0,0,0,0,0,0,-0.458757,0.000012,-0.475399,-0.627086,0
3,-0.250214,-0.369338,-0.561601,-1.395827,-1.108422,-0.267440,-3.219428,-0.069010,-0.032451,-0.133732,-0.691973,-0.416955,0.417897,-0.189182,0,1,0,0,0,0,0,0,0,0.137625,0.000012,0.072706,0.929573,0
4,0.040100,-0.158102,-0.131921,-0.001947,-0.120972,-0.848126,-2.019658,0.197624,0.083145,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,0,1,0,0,0,0,0,0,0,1.138507,0.000012,-0.051897,-0.121450,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809242,0.378541,0.031131,-0.226340,-3.996565,-0.561233,-0.848126,-4.587808,-0.825480,0.083145,-0.695958,-0.691973,-2.775520,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-1.542430,0.000012,-1.303793,-0.918843,0
809243,-0.064477,0.204919,-0.100046,-0.460184,-0.679174,-0.848126,-0.180070,0.091759,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,0,1,0,0,0,0,0,0,0,-0.057485,0.000012,0.065131,-0.660154,0
809244,-0.739584,-0.470880,-1.090445,-0.280887,-0.938102,0.759545,-2.776769,-0.839981,-0.032451,-0.133732,-0.402387,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-0.397298,0.000012,-0.530829,-0.060826,0
809245,-0.166186,-0.063051,0.502644,-0.137787,-0.347735,-0.848126,-2.776769,0.132012,0.083145,-0.133732,-0.691973,-0.416955,0.417897,-0.214837,1,0,0,0,0,0,0,0,0,-0.057485,0.000012,0.065131,-0.649813,0


# Model Training

See Project summary for types of models

In [228]:
from sklearn.model_selection import train_test_split

# Split the data into features and target variable
X = data.drop(columns=['MIS_Status_woe'])
Y = data[['MIS_Status_woe']]

# Split the data into train and test datasets (80/20 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Split the training data into train and validation datasets (60/20 split)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)

In [229]:
X_train

,City_woe,State_woe,Zip_woe,Bank_woe,BankState_woe,NAICS_woe,Term_woe,NoEmp_woe,NewExist_woe,CreateJob_woe,RetainedJob_woe,FranchiseCode_woe,UrbanRural_woe,RevLineCr_woe,LowDoc_N_woe,LowDoc_Y_woe,LowDoc_A_woe,LowDoc_0_woe,LowDoc_R_woe,LowDoc_0#_woe,LowDoc_S_woe,LowDoc_C_woe,LowDoc_1_woe,DisbursementGross_woe,BalanceGross_woe,GrAppv_woe,SBA_Appv_woe
718614,-0.057534,0.001280,0.033840,1.226106,-0.052930,0.056856,2.267246,0.193586,0.083145,-0.133732,0.635497,1.757704,0.417897,0.466279,1,0,0,0,0,0,0,0,0,0.000000,0.000012,0.728091,0.721786
792650,0.148266,0.316035,0.295849,-0.194357,0.295860,0.451162,-0.637252,-0.423380,-0.032451,-0.133732,-0.691973,-1.572283,-1.017004,-0.214837,0,1,0,0,0,0,0,0,0,-0.264084,0.000012,0.355185,-0.604023
689401,0.044535,0.162088,0.344640,-0.252089,0.032455,-0.848126,0.812396,0.015596,0.083145,-0.133732,-0.691973,0.890378,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.339339,0.000012,-0.422797,-0.681652
482874,-0.484562,-0.524615,-0.355065,-0.397298,0.147750,-0.849283,-2.776769,-0.397513,-0.032451,-0.133732,-0.691973,-0.416955,0.417897,-0.214837,1,0,0,0,0,0,0,0,0,-0.586092,0.000012,-0.984363,0.139845
50843,0.582183,0.063209,-0.090679,1.747304,0.288589,-0.848126,-2.776769,-1.085449,-0.032451,-0.133732,-0.792746,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-0.204270,0.000012,0.104580,-0.916691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414600,-0.397298,-0.369338,-0.094537,-0.675079,-1.108422,0.815170,-0.637252,0.372378,0.083145,0.352273,0.635497,0.890378,0.077680,0.466279,1,0,0,0,0,0,0,0,0,0.450000,0.000012,0.970448,0.021199
186765,0.841884,-0.282836,-0.648612,0.250927,0.147750,-0.848126,-3.219428,0.197624,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,0,1,0,0,0,0,0,0,0,-0.401795,0.000012,-0.034436,-0.800596
327870,0.073159,0.211928,-0.271720,-1.319287,-1.144512,-0.848126,-2.019658,-0.201668,-0.032451,-0.133732,-0.691973,-0.416955,0.417897,-0.189182,0,1,0,0,0,0,0,0,0,0.280930,0.000012,0.591010,-0.455023
520623,-0.269815,-0.524615,-0.640028,-1.478917,-1.467368,-0.393139,-2.019658,-0.435079,0.083145,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,0.280930,0.000012,0.591010,-0.519386


In [230]:
X_test

,City_woe,State_woe,Zip_woe,Bank_woe,BankState_woe,NAICS_woe,Term_woe,NoEmp_woe,NewExist_woe,CreateJob_woe,RetainedJob_woe,FranchiseCode_woe,UrbanRural_woe,RevLineCr_woe,LowDoc_N_woe,LowDoc_Y_woe,LowDoc_A_woe,LowDoc_0_woe,LowDoc_R_woe,LowDoc_0#_woe,LowDoc_S_woe,LowDoc_C_woe,LowDoc_1_woe,DisbursementGross_woe,BalanceGross_woe,GrAppv_woe,SBA_Appv_woe
770760,0.057958,-0.229062,-0.236955,-1.478917,-1.467368,-0.848126,-4.587808,-0.141296,-0.032451,-0.133732,-0.691973,0.890378,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.747131,0.000012,-0.790381,-0.760990
684170,-1.225928,-0.895242,-0.977116,-1.424946,0.032455,-0.616351,-4.587808,0.327251,-0.032451,-0.133732,-0.691973,-0.416955,0.417897,-0.214837,1,0,0,0,0,0,0,0,0,-1.259786,0.000012,-1.239481,0.162318
628352,0.000000,-0.164853,0.344640,-5.761938,-1.144512,-0.004662,-4.587808,-0.259558,-0.032451,0.352273,0.214285,-0.416955,0.077680,-0.189182,1,0,0,0,0,0,0,0,0,-0.516523,0.000012,-0.473671,-2.670895
464069,-0.064477,0.204919,0.129528,-0.252089,0.032455,0.305066,-0.690421,0.193586,0.083145,-0.133732,0.635497,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-0.208899,0.000012,-0.234297,0.051803
624702,-0.098507,0.147356,-0.397298,0.133271,-0.120972,-0.206779,-0.180070,0.327251,0.083145,-0.133732,0.694062,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-0.157175,0.000012,0.190984,0.398204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712540,-0.375218,-0.268316,0.152749,-0.860822,-0.958655,-0.848126,-3.219428,-0.635832,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.189182,0,1,0,0,0,0,0,0,0,-0.264084,0.000012,0.355185,-0.604023
459583,-0.452868,-0.487605,-1.542430,-0.102790,-0.582601,0.251194,-0.651441,0.327251,0.083145,-0.133732,-0.691973,-0.416955,0.077680,-0.189182,0,1,0,0,0,0,0,0,0,0.409178,0.000012,0.249329,-0.156136
212948,0.582183,0.063209,0.922476,-6.590828,0.288589,0.461411,-4.587808,-0.326690,-0.032451,0.075234,0.089317,-0.416955,0.417897,-0.189182,1,0,0,0,0,0,0,0,0,-0.488270,0.000012,-0.977116,-1.284601
254913,-0.893735,0.154977,-1.016337,0.049149,-0.052930,-0.732648,-3.219428,0.193586,-0.032451,0.693058,0.635497,-0.416955,0.417897,0.466279,1,0,0,0,0,0,0,0,0,0.000000,0.000012,0.104580,0.651687


In [231]:
X_val

,City_woe,State_woe,Zip_woe,Bank_woe,BankState_woe,NAICS_woe,Term_woe,NoEmp_woe,NewExist_woe,CreateJob_woe,RetainedJob_woe,FranchiseCode_woe,UrbanRural_woe,RevLineCr_woe,LowDoc_N_woe,LowDoc_Y_woe,LowDoc_A_woe,LowDoc_0_woe,LowDoc_R_woe,LowDoc_0#_woe,LowDoc_S_woe,LowDoc_C_woe,LowDoc_1_woe,DisbursementGross_woe,BalanceGross_woe,GrAppv_woe,SBA_Appv_woe
305782,0.260758,0.571486,1.063105,1.201362,1.069163,0.643600,-3.219428,0.015596,-0.032451,-0.133732,0.482443,0.890378,0.417897,-0.214837,1,0,0,0,0,0,0,0,0,0.280930,0.000012,0.591010,0.860762
590143,-0.060826,0.571486,-1.728532,-0.183999,-0.120684,-0.848126,-4.587808,0.197624,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.741394,0.000012,-0.826293,0.369957
22687,-0.474259,0.063209,-0.466291,-1.111005,0.288589,-0.848126,-1.480239,0.327251,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,0.000000,0.000012,-1.173622,-0.702679
35394,-0.397298,0.090475,0.000000,2.554341,0.288589,-0.876981,1.098874,-0.141296,-0.032451,-0.133732,0.348379,0.890378,0.417897,-0.214837,1,0,0,0,0,0,0,0,0,0.770752,0.000012,0.756378,1.814293
63295,0.000000,0.316035,0.000000,1.037787,0.295860,-0.848126,-0.756409,0.091759,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.475293,0.000012,-0.501163,-0.728988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644353,-0.397298,-0.664123,-0.602423,-1.960442,-0.999530,-0.848126,-4.587808,0.197624,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.403731,0.000012,-0.270841,-0.891174
807226,-0.648612,-0.369338,-0.139963,-1.765574,-1.108422,-0.279628,-2.019658,0.197624,0.083145,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,0,1,0,0,0,0,0,0,0,-0.204270,0.000012,0.104580,-0.814127
770558,1.180888,0.147356,0.855465,0.579143,0.668106,0.466550,-3.219428,-0.277641,-0.032451,0.075234,0.212934,-0.416955,0.417897,0.466279,1,0,0,0,0,0,0,0,0,0.280930,0.000012,0.591010,0.860762
287116,0.057958,-0.229062,-0.236955,-0.252089,-0.924157,-0.848126,-0.637252,-0.554480,-0.032451,-0.133732,-0.691973,-0.416955,-1.017004,-0.214837,1,0,0,0,0,0,0,0,0,-0.057485,0.000012,0.065131,-0.649813


In [232]:
from sklearn.decomposition import PCA

# Create a PCA object with the desired number of components
pca_obj = PCA(n_components=5)

# Fit the PCA model and apply the dimensionality reduction to the training data
X_train_pca = pca_obj.fit_transform(X_train)

# Apply the dimensionality reduction to the validation and test datasets using the transform method
X_val_pca = pca_obj.transform(X_val)
X_test_pca = pca_obj.transform(X_test)

In [233]:
# using GLM for another five features
# from the data card
# I have considered the following 5 features for thee GLM model because they are expected to have a significant impact on the loan status. I have explanined below why I choose them.
# 1. Term: The loan term is expected to be a significant factor in determining loan status as longer loan terms may lead to higher default rates.
# 2. SBA_Appv: The amount of the loan that is guaranteed by the SBA is expected to impact the likelihood of loan default.
# 3. GrAppv: The gross amount of the loan approved by the bank is expected to impact the likelihood of loan default.
# 4. UrbanRural: The borrower's location (urban or rural) is expected to impact the likelihood of loan default due to varying economic conditions and resources.
# 5. NoEmp: The number of business employees may impact the likelihood of loan default as businesses with more employees may have more resources to pay off the loan. 

import pandas as pd
import statsmodels.api as sm

# Define the five features to be used in the GLM model
glm_features = ['Term_woe', 'SBA_Appv_woe', 'CreateJob_woe', 'NoEmp_woe', 'GrAppv_woe']

# Define the formula for the GLM
glm_formula = 'MIS_Status_woe ~ Term_woe + SBA_Appv_woe + CreateJob_woe + NoEmp_woe + GrAppv_woe'

# Create a new DataFrame with X_train and Y_train combined
train_glm = pd.concat([X_train, Y_train], axis=1)

# Create and fit the GLM model
glm_model = sm.GLM.from_formula(formula=glm_formula, data=train_glm).fit()

# Predict using GLM model for train, validation, and test datasets
X_train_pred_glm = glm_model.predict(X_train[glm_features])
X_val_pred_glm = glm_model.predict(X_val[glm_features])
X_test_pred_glm = glm_model.predict(X_test[glm_features])

In [234]:
# Define a function to add PCA and GLM columns to a given dataset
def add_pca_glm_columns(dataset, pca_data, glm_data):
     # Loop through the PCA columns and add them to the dataset
    for i in range(5):
        dataset[f"pca{i+1}"] = pca_data[:, i]
    
    # Add the GLM column to the dataset
    dataset["GLM1"] = glm_data
        
    # Loop through the GLM features, multiply them by the GLM column, and add the results to the dataset
    for i, feature in enumerate(glm_features):
        dataset[f"GLM{i+2}"] = glm_data * dataset[feature]

# Add PCA and GLM columns to X_train, X_val, and X_test
add_pca_glm_columns(X_train, X_train_pca, X_train_pred_glm)
add_pca_glm_columns(X_val, X_val_pca, X_val_pred_glm)
add_pca_glm_columns(X_test, X_test_pca, X_test_pred_glm)

## Model Tuning

Choose one model from the above list. You should provide reasoning on why you have picked the model over others. Perform tuning for the selected model:
- Hyper-parameter tuning. Your hyper-parameter search space should have at least 50 combinations.
- To avoid overfitting and provide you with reasonable estimate of model performance on hold-out dataset, you will need to split your dataset as following:
    - Train, will be used to train model
    - Validation, will be used to validate model each round of training
    - Testing, will be used to provide final performance metrics, used only once on the final model
- Feature engineering. See project description

**Selelct final model that produces best performance on the Test dataset.**
- For the best model, calculate probability threshold to maximize F1. 

In [235]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define hyperparameter values to search over for the regularization parameter C.
# I used a logarithmic scale with 11 values ranging from 0.0001 to 10000.
C = np.logspace(-4, 4, 11)

# Define penalty values to search over for the logistic regression model.
penalty = ["l1", "l2", "elasticnet", "none"]

# Define solver values to search over for the logistic regression model.
solver = ['liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

# Initialize variables for tracking accuracy scores and hyperparameters
comb = 0  # counter for number of hyperparameter combinations tested
acc1, acc2 = [], []  # empty lists to hold training and validation accuracy scores
all_params = []  # empty list to hold all hyperparameter combinations tested

# Loop over all possible hyperparameter combinations
for c_value in C:
    for penalty_type in penalty:
        for solver_type in solver:
            params = [c_value, penalty_type, solver_type]  # current set of hyperparameters
            comb += 1  # increment combination counter
            try:
                # Train a logistic regression model using the current set of hyperparameters
                logreg = LogisticRegression(C=c_value, penalty=penalty_type, solver=solver_type)
                logreg.fit(X_train, Y_train)
                
                # Use the trained model to make predictions on the training and validation sets
                Y_train_pred = logreg.predict(X_train)
                Y_val_pred = logreg.predict(X_val)
                
                # Calculate the accuracy scores and add them to the appropriate lists
                a1 = accuracy_score(Y_train, Y_train_pred)
                a2 = accuracy_score(Y_val, Y_val_pred)
                acc1.append(a1)
                acc2.append(a2)
                all_params.append(params)
                
                # Print information about the current combination, its hyperparameters, and its accuracy scores
                print(f"combination {comb}, parameters {params}, train & val accuracy {a1} & {a2}")
            except Exception as e:
                # If the current set of hyperparameters results in an error, skip it and continue to the next one
                print(f"combination {comb}, parameters {params}, invalid parameters")

# Find the hyperparameters that produced the best validation accuracy score
best_accuracy_index = np.argmax(acc2)
best_hyperparams = all_params[best_accuracy_index]

# Train a new logistic regression model using the best hyperparameters and evaluate it on the training and test sets
logreg = LogisticRegression(C=best_hyperparams[0], penalty=best_hyperparams[1], solver=best_hyperparams[2])
logreg.fit(X_train, Y_train)
Y_train_pred = logreg.predict(X_train)
Y_test_pred = logreg.predict(X_test)
print("Train accuracy_score:", accuracy_score(Y_train, Y_train_pred))
print("Test accuracy_score:", accuracy_score(Y_test, Y_test_pred))

# Calculate the predicted probabilities of the positive class (i.e., class 1) for the test set
Y_pred_test_prob = logreg.predict_proba(X_test)[:, 1]

# Use a range of probability thresholds to calculate F1 scores for each threshold
thresholds = np.arange(0, 1, 0.01)
f1_scores = [f1_score(Y_test, Y_pred_test_prob >= t, average='macro') for t in thresholds]

# Find the probability threshold that maximizes the F1 score
opt_f1_thresh = thresholds[np.argmax(f1_scores)]

# Print the maximum F1 score and the corresponding optimal threshold
print("Maximum F1 score:", np.max(f1_scores))
print("Optimal threshold:", opt_f1_thresh)

combination 1, parameters [0.0001, 'l1', 'liblinear'], train & val accuracy 0.9280831721748873 & 0.9280321285140563
combination 2, parameters [0.0001, 'l1', 'newton-cg'], invalid parameters
combination 3, parameters [0.0001, 'l1', 'newton-cholesky'], invalid parameters
combination 4, parameters [0.0001, 'l1', 'sag'], invalid parameters
combination 5, parameters [0.0001, 'l1', 'saga'], train & val accuracy 0.9284126974319685 & 0.9284646277417362
combination 6, parameters [0.0001, 'l2', 'liblinear'], train & val accuracy 0.9345871769365273 & 0.934686438059932
combination 7, parameters [0.0001, 'l2', 'newton-cg'], train & val accuracy 0.9344162357094163 & 0.9345319740500463
combination 8, parameters [0.0001, 'l2', 'newton-cholesky'], train & val accuracy 0.9344162357094163 & 0.9345319740500463
combination 9, parameters [0.0001, 'l2', 'sag'], train & val accuracy 0.9344162357094163 & 0.9345319740500463
combination 10, parameters [0.0001, 'l2', 'saga'], train & val accuracy 0.93441623570941

combination 77, parameters [0.025118864315095822, 'none', 'newton-cg'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 78, parameters [0.025118864315095822, 'none', 'newton-cholesky'], train & val accuracy 0.9366755432532793 & 0.9360457213469262
combination 79, parameters [0.025118864315095822, 'none', 'sag'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 80, parameters [0.025118864315095822, 'none', 'saga'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 81, parameters [0.15848931924611143, 'l1', 'liblinear'], train & val accuracy 0.9366837813847063 & 0.9360951498300896
combination 82, parameters [0.15848931924611143, 'l1', 'newton-cg'], invalid parameters
combination 83, parameters [0.15848931924611143, 'l1', 'newton-cholesky'], invalid parameters
combination 84, parameters [0.15848931924611143, 'l1', 'sag'], invalid parameters
combination 85, parameters [0.15848931924611143, 'l1', 'saga'], train & val acc

combination 157, parameters [39.81071705534978, 'none', 'newton-cg'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 158, parameters [39.81071705534978, 'none', 'newton-cholesky'], train & val accuracy 0.9366755432532793 & 0.9360457213469262
combination 159, parameters [39.81071705534978, 'none', 'sag'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 160, parameters [39.81071705534978, 'none', 'saga'], train & val accuracy 0.9366940790489902 & 0.9360642570281125
combination 161, parameters [251.18864315095823, 'l1', 'liblinear'], train & val accuracy 0.9366899599832766 & 0.9360642570281125
combination 162, parameters [251.18864315095823, 'l1', 'newton-cg'], invalid parameters
combination 163, parameters [251.18864315095823, 'l1', 'newton-cholesky'], invalid parameters
combination 164, parameters [251.18864315095823, 'l1', 'sag'], invalid parameters
combination 165, parameters [251.18864315095823, 'l1', 'saga'], train & val accuracy 0.

Note: We select the model with a high accuracy score on the validation set because it needs to exhibit strong performance on data that it has not previously encountered.

## Save all artifacts

Save all artifacts needed for scoring function:
- Trained model
- Encoders
- Any other arficats you will need for scoring

**You should stop your notebook here. Scoring function should be in a separate file/notebook.**

## Stop Here. Create new file/notebook

## ==============================================

In [237]:
import os
import pickle

# Print the current working directory
print(os.getcwd())

C:\Users\nagal\virtualenv\Project-1


In [238]:
# Check if the 'artifacts' directory exists, if not, create it
if not os.path.exists('./artifacts'):
    os.makedirs('./artifacts')

# Dump the models and encoders into pickle files in the 'artifacts' directory
if logreg is not None:
    pickle.dump(logreg, open('./artifacts/LogisticRegressionModel.pkl', 'wb'))
else:
    print('Logistic regression model is not available.')

if one_hot_encoder is not None:
    pickle.dump(one_hot_encoder, open('./artifacts/one_hot_encoder.pkl', 'wb'))
else:
    print('one_hot_encoderr is not available.')

if woe_encoder is not None:
    pickle.dump(woe_encoder, open('./artifacts/woe_encoder.pkl', 'wb'))
else:
    print('WOE encoder is not available.')

if pca_obj is not None:
    pickle.dump(pca_obj, open('./artifacts/pca_obj.pkl', 'wb'))
else:
    print('PCA model is not available.')

if glm_model is not None:
    pickle.dump(glm_model, open('./artifacts/glm.pkl', 'wb'))
else:
    print('GLM model is not available.')

## Model Scoring

Write function that will load artifacts from above, transform and score on a new dataset.
Your function should return Python list of labels. For example: [0,1,0,1,1,0,0]


In [6]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return pandas DF with following columns:
            - index
            - label
            - probability_0
            - probability_1
    """
    pass

### Example of Scoring function

Don't copy the code as is. It is provided as an example only. 
- Function `train_model` - you need to focus on model and artifacts saving:
    ```
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    ```
- Function `project_1_scoring` - you should have similar function with name `project_1_scoring`. The function will:
    - Get Pandas dataframe as parameter
    - Will load model and all needed encoders
    - Will perform needed manipulations on the input Pandas DF - in the exact same format as input file for the project, minus MIS_Status feature
    - Return Pandas DataFrame
        - record index
        - predicted class for threshold maximizing F1
        - probability for class 0 (PIF)
        - probability for class 1 (CHGOFF)


In [7]:
"""
Don't copy of use the cell code in any way!!!
The code is provided as an example of generating artifacts for scoring function
Your scoring function code should not have model training part!!!!
"""
import pandas as pd
import numpy as np
def train_model(data):
    """
    Train sample model and save artifacts
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    from sklearn.impute import SimpleImputer
    
    target_col = "Survived"
    cols_to_drop = ['Name', 'Ticket', 'Cabin','SibSp', 'Parch', 'Sex','Embarked','PassengerId','Survived']
    y = data[target_col]
    X = data.drop(columns=[target_col])
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    imp_age_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_age_mean.fit(X[['Age']])
    X['Age'] = imp_age_mean.transform(X[['Age']])


    ohe_orig_columns = ["Embarked","Sex"]
    cat_encoders = {}
    for col in ohe_orig_columns:
        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        enc.fit(X[[col]])
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X= pd.concat([X, result_train], axis=1)
        cat_encoders[col] = [deepcopy(enc),"ohe"]
        
    clf = LogisticRegression(max_iter=1000, random_state=0)
    
    columns_to_train = [x for x in X.columns if x not in cols_to_drop]
    clf.fit(X[columns_to_train], y)
    
    # Todo: Add code to calculate optimal threshold. Replace 0.5 !!!!!
    threshold = 0.5
    # End Todo
    
    artifacts_dict = {
        "model": clf,
        "cat_encoders": cat_encoders,
        "imp_age_mean": imp_age_mean,
        "ohe_columns": ohe_orig_columns,
        "columns_to_train":columns_to_train,
        "threshold": threshold
    }
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "wb")
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    
    artifacts_dict_file.close()    
    return clf

In [8]:
df_train = pd.read_csv('titanic.csv')
train_model(df_train)

LogisticRegression(max_iter=1000, random_state=0)

### Example scoring function

This is example only. Don't copy code as is!!!   
You must place scoring function in a separate Python file or Jupyter notebook.   

**Don't place function in the same notebook as rest of the code**

In [9]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    
    X = data.copy()
    
    '''Load Artifacts'''
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    
    clf = artifacts_dict["model"]
    cat_encoders = artifacts_dict["cat_encoders"]
    imp_age_mean = artifacts_dict["imp_age_mean"]
    ohe_columns = artifacts_dict["ohe_columns"]
    columns_to_score = artifacts_dict["columns_to_train"]
    threshold = artifacts_dict["threshold"]
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    X['Age'] = imp_age_mean.transform(X[['Age']])
    
    '''Encode categorical columns'''
    for col in ohe_columns:
        enc = cat_encoders[col][0]
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X = pd.concat([X, result_train], axis=1)
        
    y_pred_proba = clf.predict_proba(X[columns_to_score])
    y_pred = (y_pred_proba[:,0] < threshold).astype(np.int16)
    d = {"index":data["PassengerId"],
         "label":y_pred,
         "probability_0":y_pred_proba[:,0],
         "probability_1":y_pred_proba[:,1]}
    
    return pd.DataFrame(d)

In [10]:
project_1_scoring(df_train).head()

,index,label,probability_0,probability_1
0,1,0,0.901298,0.098702
1,2,1,0.071879,0.928121
2,3,1,0.367665,0.632335
3,4,1,0.098564,0.901436
4,5,0,0.923460,0.076540
